In [1]:
import pandas as pd
import numpy as np
import re
import os

# Set the path as tangible
os.chdir('..')

# Check current path 
print(os.getcwd())


c:\Users\musta\Documents\Python Projects\tangible


In [2]:
# Read the text file
file_path = 'data/contractions/102-Ahmet Ümit _ İstanbul Hatırası.txt'
base_name = os.path.basename(file_path)
document_name = os.path.splitext(base_name)[0]
print("Document: " + document_name)


with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()


Document: 102-Ahmet Ümit _ İstanbul Hatırası


In [2]:
# 1) Küçük harfe dönüştür
text = text.lower()

# 2) Noktalama işaretlerini sil (regex ile)
text = re.sub(r'[^\w\s]', '', text)

# 3) Sayı vb. temizle
text = re.sub(r'\d+', '', text)

# Tokenization
tokens = text.split()  # Boşluk karakterine göre ayırır


NameError: name 'text' is not defined

In [2]:
# Create data frame
df = pd.DataFrame(tokens, columns=['word'])
df['count'] = 1

# Gruplayarak frekans (count) değerini hesapla
word_counts = df.groupby('word')['count'].sum().reset_index()
# Dosya ismini ekle
word_counts['source'] = document_name

# Frekansına göre büyükten küçüğe sırala
word_counts = word_counts.sort_values('count', ascending=False)

# İlk 20 kelimeyi görüntüleyelim
print(word_counts.head(20))


NameError: name 'tokens' is not defined

In [21]:
# CSV dosyasına kaydet
csv_path = 'data/contractions/word_frequencies.csv'
if not os.path.exists(csv_path):
    # CSV yoksa oluştur (header dahil)
    word_counts.to_csv(csv_path, index=False, encoding='utf-8')
else:
    # CSV varsa oku ve birleştir (append)
    existing_df = pd.read_csv(csv_path)
    combined_df = pd.concat([existing_df, word_counts], ignore_index=True)
    combined_df.to_csv(csv_path, index=False, encoding='utf-8')


In [2]:
# Tekrar eden sözcükleri birleştir
csv_path = 'data/contractions/word_frequencies.csv'
df = pd.read_csv(csv_path)

# Sözcük bazında topla
df_merged = df.groupby('word', as_index=False)['count'].sum()
df_merged = df_merged.sort_values('count', ascending=False)

# Eğer 'source' (belge adı) sütunu da varsa ve hem 'word' hem de 'source' bazında birleştirmek istiyorsanız:
# df_merged = df.groupby(['word', 'source'], as_index=False)['count'].sum()
# Ortalamayı ve standart sapmayı hesapla
mean_val = df_merged['count'].mean()
std_val = df_merged['count'].std()

# Standart sapmanın 0 olması ihtimaline karşı kontrol et
if std_val == 0:
    # Böyle bir durumda tüm z-skorları 0 yapılabilir
    df_merged['z_score'] = 0
else:
    # (count - ortalama) / std formülüyle z-skorunu ekle
    df_merged['z_score'] = (df_merged['count'] - mean_val) / std_val

# Sonucu incele
print(mean_val)
print(std_val)
print(df_merged)

# CSV olarak kaydet
#df_merged.to_csv('data/contractions/word_frequencies_merged.csv', index=False, encoding='utf-8')


4.870993819594753
43.793707573021614
                    word  count     z_score
4821                 bir   5659  129.108251
5356                  bu   2932   66.839032
31948                 ve   2425   55.262026
7022                  de   2004   45.648773
6669                  da   1519   34.574122
...                  ...    ...         ...
38324      şüphelenmemiş      1   -0.088392
38323  şüpheleniyorsunuz      1   -0.088392
38322  şüpheleniyorsanız      1   -0.088392
38321            şüpheci      1   -0.088392
15                 abdye      1   -0.088392

[38347 rows x 3 columns]


In [16]:
# Stopwords
import nltk

# Türkçe stopword'leri indirelim (bir kereye mahsus)
# nltk.download('stopwords')
from nltk.corpus import stopwords

# NLTK'nin Türkçe stopword'lerini alalım (tamamen yeterli olmayabilir, isteğe göre genişletilebilir)
#turkish_stopwords_nltk = stopwords.words('turkish')
turkish_stopwords_nltk = []
print(turkish_stopwords_nltk)
# Ek olarak kendi özel stopword listenizi de tanımlayabilirsiniz
csv_path_stopwords = 'data/contractions/stopwords.csv'
df_my_stopwords = pd.read_csv(csv_path_stopwords, header=None)
my_stopwords = df_my_stopwords[0].tolist()
# İki listeyi birleştirelim

turkish_stopwords = set(turkish_stopwords_nltk + my_stopwords)

# --- Asıl analiz ---

# 1) CSV'yi okuyalım
csv_path = 'data/contractions/word_frequencies.csv'
df = pd.read_csv(csv_path)

# 2) Sözcük bazında topla (aynı 'word' birden çok satırda olabilir diye)
df_merged = df.groupby('word', as_index=False)['count'].sum()

# 3) Stopword’leri çıkaralım
df_stopwords_removed = df_merged[~df_merged['word'].isin(turkish_stopwords)]

# 4) Temizlenmiş veriyi frekansa göre sırala
df_stopwords_removed = df_stopwords_removed.sort_values('count', ascending=False).reset_index(drop=True)
#df_stopwords_removed = df_stopwords_removed.sort_values('word', ascending=True).reset_index(drop=True)

# 5) Ortalamayı ve standart sapmayı hesaplayalım
mean_val_sw = df_stopwords_removed['count'].mean()
std_val_sw = df_stopwords_removed['count'].std()

# 6) Z-skorunu hesapla
if std_val_sw == 0:
    df_stopwords_removed['z_score'] = 0
else:
    df_stopwords_removed['z_score'] = (df_stopwords_removed['count'] - mean_val_sw) / std_val_sw

# 7) Sonucu görelim
print(mean_val_sw)
print(std_val_sw)
print(df_stopwords_removed.head(20))

# 8) İsterseniz CSV olarak kaydedebilirsiniz
# df_stopwords_removed.to_csv('data/contractions/word_frequencies_merged_no_stopwords.csv', 
#                             index=False, encoding='utf-8')


[]
4.4000730441134275
21.36408305210143
     word  count    z_score
0     ama   1335  62.282099
1      ne   1234  57.554538
2    için   1053  49.082375
3    daha    970  45.197349
4   kadar    751  34.946500
5    diye    738  34.338002
6     çok    601  27.925370
7     her    573  26.614759
8   sonra    556  25.819031
9      en    503  23.338232
10  değil    498  23.104194
11    ben    497  23.057387
12  bütün    494  22.916964
13    iki    489  22.682927
14     mi    485  22.495696
15    hiç    473  21.934006
16   bile    441  20.436165
17   önce    437  20.248935
18  büyük    421  19.500014
19    ali    421  19.500014


In [ ]:
# Bir Harfli Kısaltmalar: Z-Skoru

# CSV'yi oku
csv_path = 'data/contractions/word_frequencies.csv'
df_single = pd.read_csv(csv_path)

# Tek harfli kısaltmalar listesi
csv_path_con_single = 'data/contractions/con_single.csv'
df_con_single = pd.read_csv(csv_path_con_single, header=None)
contractions_list_single = df_con_single[0].tolist()

# Stopwordleri çıkar
csv_path_stopwords_single = 'data/contractions/stopwords_single.csv'
df_my_stopwords_single = pd.read_csv(csv_path_stopwords_single, header=None)
my_stopwords_single = df_my_stopwords_single[0].tolist()
df_single = df_single[~df_single['word'].isin(my_stopwords_single)]
#print(df_single[df_single['word'] == 'gibi'])

# 1) Listedeki sözcüklerle BAŞLAYAN tüm sözcüklerin toplam frekansını hesapla
results = []
for cword in contractions_list_single:
    sub_count = df_single[df_single['word'].str.startswith(cword, na=False)]['count'].sum()

    # Eğer 'büyük' ve 'artık' sözcüklerinin yumuşamış formları varsa frekansa ekle
    if cword == "büyük":
        sub_count += df_single[df_single['word'].str.startswith("büyüğ", na=False)]['count'].sum()
    elif cword == "artık":
        sub_count += df_single[df_single['word'].str.startswith("artığ", na=False)]['count'].sum()

    results.append({'word': cword, 'count': sub_count})

df_single_freq = pd.DataFrame(results)

# 2) Ortalamayı ve standart sapmayı hesapla
mean_val_single = df_single_freq['count'].mean()
std_val_single = df_single_freq['count'].std()

# 3) Standart sapmanın 0 olması ihtimaline karşı kontrol yap
if std_val_single == 0:
    # Böyle bir durumda tüm z-skorlarını 0 yap
    df_single_freq['z_score'] = 0
else:
    # (count - ortalama) / std formülüyle z-skorunu ekleyelim
    df_single_freq['z_score'] = (df_single_freq['count'] - mean_val_single) / std_val_single

# 4) Log-Transfer uygula

# 'count' değerleri için log(1+x) dönüşümü
df_single_freq['log_count'] = np.log1p(df_single_freq['count'])  # log(count + 1)

# 5) Log değerlerin ortalamasını ve std'sini alın
mean_log = df_single_freq['log_count'].mean()
std_log = df_single_freq['log_count'].std()

# 6) Log değer üzerinden z-skoru
if std_log == 0:
    df_single_freq['z_score_log'] = 0
else:
    df_single_freq['z_score_log'] = (df_single_freq['log_count'] - mean_log) / std_log

# 7) Z-Skoruna göre kategorilere ayır
# Kategoriler için bin aralıkları (soldan sağa sıralı):
bins = [-np.inf, -1, -0.5, 0, 0.5, 1, np.inf]
# Bu aralıklara denk gelen etiketler (labels):
labels = ["çok düşük", "düşük", "orta düşük", "orta yüksek", "yüksek", "çok yüksek"]
df_single_freq['z_categories'] = pd.cut(df_single_freq['z_score_log'], bins=bins, labels=labels)

# 8) Yüzdelik hesapla
# Toplam 'count' değerini hesapla
total_count_single = df_single_freq['count'].sum()
# Yeni sütun: 'percentage' (yüzde)
df_single_freq['percentage'] = (df_single_freq['count'] / total_count_single) * 100
df_single_percent = df_single_freq[['word', 'count', 'percentage', 'z_score_log', 'z_categories']]

# 9) Sırala ve sonucu yazdır
df_single_percent = df_single_percent.sort_values("count", ascending=False).reset_index(drop=True)
print(mean_val_single)
print(std_val_single)
print(df_single_percent)
# Çok düşük olanları sırala
#print(df_single_freq[df_single_freq['z_categories'] == 'yüksek'])
#print(df_single_percent[df_single_percent['word'] == 'büyük'])


Empty DataFrame
Columns: [word, count, source]
Index: []
402.42857142857144
286.86122965532496
       word  count  percentage  z_score_log z_categories
0      daha    976    8.661697     0.975982       yüksek
1       var    945    8.386581     0.953657       yüksek
2     kadar    797    7.073127     0.835857       yüksek
3       her    765    6.789137     0.807520       yüksek
4       yok    736    6.531771     0.780797       yüksek
5       çok    637    5.653177     0.680920       yüksek
6       gün    617    5.475683     0.658867       yüksek
7       şey    613    5.440185     0.654371       yüksek
8     sonra    609    5.404686     0.649846       yüksek
9     zaman    554    4.916578     0.584421       yüksek
10     için    543    4.818956     0.570560       yüksek
11    büyük    469    4.162229     0.469321  orta yüksek
12     öyle    379    3.363507     0.322145  orta yüksek
13    artık    349    3.097267     0.265204  orta yüksek
14     göre    331    2.937522     0.228646  orta 

In [39]:
# İki Harfli Kısaltmalar: Z-Skoru

# CSV'yi oku
csv_path = 'data/contractions/word_frequencies.csv'
df = pd.read_csv(csv_path)

# İki harfli kısaltmalar listesi
csv_path_cons_double = 'data/contractions/cons_double.csv'
df_cons_double = pd.read_csv(csv_path_cons_double, header=None)
# 2. sütundaki değerleri listeye al
contractions_double = df_cons_double[1].tolist()
print(contractions_double)

# 1) Listedeki her sözcükle BAŞLAYAN tüm sözcüklerin toplam frekansını bulalım
results = []
for cword in contractions_double:
    sub_count = df[df['word'].str.startswith(cword, na=False)]['count'].sum()
    # Eğer sözcüklerin yumuşamış formları varsa frekansa ekle
    if cword == "çocuk":
        sub_count += df[df['word'].str.startswith("çocuğ", na=False)]['count'].sum()
    elif cword == "kitap":
        sub_count += df[df['word'].str.startswith("kitab", na=False)]['count'].sum()
    elif cword == "küçük":
        sub_count += df[df['word'].str.startswith("küçüğ", na=False)]['count'].sum()
    elif cword == "sıcak":
        sub_count += df[df['word'].str.startswith("sıcağ", na=False)]['count'].sum()
    elif cword == "soğuk":
        sub_count += df[df['word'].str.startswith("soğuğ", na=False)]['count'].sum()
    elif cword == "toprak":
        sub_count += df[df['word'].str.startswith("toprağ", na=False)]['count'].sum()

    results.append({'word': cword, 'count': sub_count})

df_double_freq = pd.DataFrame(results)

# 2) Ortalamayı ve standart sapmayı hesapla
mean_val_double = df_double_freq['count'].mean()
std_val_double = df_double_freq['count'].std()

# 3) Standart sapmanın 0 olması ihtimaline karşı kontrol yap
if std_val_double == 0:
    df_double_freq['z_score'] = 0
else:
    # (count - ortalama) / std formülüyle z-skorunu ekleyelim

    df_double_freq['z_score'] = (df_double_freq['count'] - mean_val_double) / std_val_double

# 4) Log-Transfer uygula
# 'count' değerleri için log(1+x) dönüşümü
df_double_freq['log_count'] = np.log1p(df_double_freq['count'])  # log(count + 1)

# 5) Log değerlerin ortalamasını ve std'sini alın
mean_double_log = df_double_freq['log_count'].mean()
std_double_log = df_double_freq['log_count'].std()

# 6) Log değer üzerinden z-skoru
if std_double_log == 0:
    df_double_freq['z_score_log'] = 0
else:
    df_double_freq['z_score_log'] = (df_double_freq['log_count'] - mean_double_log) / std_double_log

# 7) Z-Skoruna göre kategorilere ayır
# Kategoriler için bin aralıkları (soldan sağa sıralı):
bins = [-np.inf, -1, -0.5, 0, 0.5, 1, np.inf]
# Bu aralıklara denk gelen etiketler (labels):
labels = ["çok düşük", "düşük", "orta düşük", "orta yüksek", "yüksek", "çok yüksek"]
df_double_freq['z_categories'] = pd.cut(df_double_freq['z_score_log'], bins=bins, labels=labels)

# 8) Yüzdelik hesapla
# Toplam 'count' değerini hesapla
total_count_double = df_double_freq['count'].sum()
# Yeni sütun: 'percentage' (yüzde)
df_double_freq['percentage'] = (df_double_freq['count'] / total_count_double) * 100
df_double_percent = df_double_freq[['word', 'count', 'percentage', 'z_score_log', 'z_categories']]


# 9) Sırala ve sonucu yazdır
df_double_percent = df_double_percent.sort_values("count", ascending=False).reset_index(drop=True)
print(mean_val_double)
print(std_val_double)
#print(df_double_percent)
# Çok düşük olanları sırala
print(df_double_percent[df_double_percent['z_categories'] == 'çok düşük'])
#print(df_double_percent[df_double_percent['word'] == 'küçük'])


['beden', 'bağımsız', 'bilgi', 'bundan', 'beraber', 'basit', 'başka', 'bütün', 'böyle', 'bazı', 'cumhuriyet', 'cisim', 'cevap', 'çocuk', 'çoğu', 'çalışkan', 'çünkü', 'çeşit', 'çevre', 'değil', 'dolayı', 'demek', 'deney', 'devlet', 'fazla', 'gibi', 'gece', 'genel', 'güzel', 'haber', 'hiç', 'halk', 'hemen', 'hangi', 'hepsi', 'hayat', 'hazır', 'kabul', 'küçük', 'kadın', 'kahraman', 'kolay', 'kendi', 'kısım', 'kitap', 'kuvvet', 'kuzey', 'lazım', 'madde', 'millet', 'memleket', 'ancak', 'niçin', 'anadolu', 'nehir', 'anne', 'insan', 'arka', 'örneğin', 'arası', 'orta', 'sebep', 'sıcak', 'soğuk', 'eski', 'osmanlı', 'soru', 'savaş', 'aşağı', 'şehir', 'şekil', 'şimdi', 'işte', 'şöyle', 'tabiat', 'toprak', 'vatan', 'aydın', 'yukarı', 'yalnız', 'yeni', 'yarar', 'yavaş', 'yüzyıl', 'özel', 'uzun', 'üzere']
158.41379310344828
225.64592307027345
        word  count  percentage  z_score_log z_categories
71     kuzey     13    0.094326    -1.021864    çok düşük
72   beraber     12    0.087070    -1.07054